In [1]:
import sys
sys.path.append('..')

In [2]:
import tagnews

In [3]:
df = tagnews.load_data()

..\tagnews\utils\load_data.py:194: RuntimeWarning: 1 location strings were not found in the bodytext.
  RuntimeWarning)


In [4]:
import pandas as pd

In [5]:
time = pd.to_datetime(df['created'])
df = df.loc[('2015' < time) & (time < '2017')]

In [6]:
crimetags = tagnews.CrimeTags()
geoextractor = tagnews.GeoCoder()

C:\Users\kevin.rose\AppData\Local\Continuum\Anaconda2\envs\cjp\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\kevin.rose\AppData\Local\Continuum\Anaconda2\envs\cjp\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\kevin.rose\AppData\Local\Continuum\Anaconda2\envs\cjp\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator OneVsRestClassifier from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\kevin.rose\AppData\Local\Continuum\Anaconda2\envs\cjp\lib\site-packages\sklearn\base.py:311: U

In [7]:
preds = df['bodytext'].apply(crimetags.tagtext_proba)

In [8]:
preds.shape

(78570, 38)

In [9]:
preds.columns

Index(['ARSN', 'BEAT', 'BURG', 'CCCC', 'CCJ', 'CCSP', 'CPBD', 'CPD', 'CPLY',
       'CPS', 'CPUB', 'DOMV', 'DRUG', 'DUI', 'ENVI', 'FRUD', 'GANG', 'GLBTQ',
       'GUNV', 'HOMI', 'IDOC', 'ILSC', 'ILSP', 'IMMG', 'IPRA', 'JUVE', 'OEMC',
       'PARL', 'POLB', 'POLM', 'PROB', 'REEN', 'ROBB', 'SAO', 'SEXA', 'TASR',
       'UNSPC', 'VIOL'],
      dtype='object')

In [10]:
(preds['BURG'] > 0.7).sum()

275

In [11]:
(preds['ARSN'] > 0.7).sum()

157

In [12]:
geostrings = df.loc[preds['BURG'] > 0.7, 'bodytext'].apply(geoextractor.extract_geostrings)

In [13]:
sum(geostrings.apply(len))

729

In [14]:
lat_longs = []
for i, gs in enumerate(geostrings):
    lat_longs.append(geoextractor.lat_longs_from_geostring_lists(gs, sleep_secs=5))
    if not (i % 70):
        print(i / geostrings.size)

0.0
0.2545454545454545


Status code 401 from https://services.gisgraphy.com/geocoding/: ERROR - 401 Client Error: Unauthorized for url: https://services.gisgraphy.com/geocoding/?address=400+Keller+Avenue.+Chicago+Illinois&limitnbresult=1&format=json
Status code 401 from https://services.gisgraphy.com/geocoding/: ERROR - 401 Client Error: Unauthorized for url: https://services.gisgraphy.com/geocoding/?address=Georgia+Chicago+Illinois&limitnbresult=1&format=json
Status code 401 from https://services.gisgraphy.com/geocoding/: ERROR - 401 Client Error: Unauthorized for url: https://services.gisgraphy.com/geocoding/?address=2100+West+110th+Street+Chicago+Illinois&limitnbresult=1&format=json
Status code 401 from https://services.gisgraphy.com/geocoding/: ERROR - 401 Client Error: Unauthorized for url: https://services.gisgraphy.com/geocoding/?address=Side+Irving+Park&limitnbresult=1&format=json
Status code 401 from https://services.gisgraphy.com/geocoding/: ERROR - 401 Client Error: Unauthorized for url: https://se

0.509090909090909


Status code 401 from https://services.gisgraphy.com/geocoding/: ERROR - 401 Client Error: Unauthorized for url: https://services.gisgraphy.com/geocoding/?address=Yards&limitnbresult=1&format=json
Status code 401 from https://services.gisgraphy.com/geocoding/: ERROR - 401 Client Error: Unauthorized for url: https://services.gisgraphy.com/geocoding/?address=block+South+Chicago+Illinois&limitnbresult=1&format=json


0.7636363636363637


Status code 401 from https://services.gisgraphy.com/geocoding/: ERROR - 401 Client Error: Unauthorized for url: https://services.gisgraphy.com/geocoding/?address=north+Chicago+Illinois&limitnbresult=1&format=json
Status code 401 from https://services.gisgraphy.com/geocoding/: ERROR - 401 Client Error: Unauthorized for url: https://services.gisgraphy.com/geocoding/?address=400+Sterling+East+Chicago+Illinois&limitnbresult=1&format=json
Status code 401 from https://services.gisgraphy.com/geocoding/: ERROR - 401 Client Error: Unauthorized for url: https://services.gisgraphy.com/geocoding/?address=Lincoln+Park+n+eighborhood+Chicago+Illinois&limitnbresult=1&format=json


In [15]:
import numpy as np

In [16]:
import folium

from folium.plugins import HeatMap

In [17]:
coords = [x for ll in lat_longs for x in ll[0] if x is not None]

In [18]:
pd.DataFrame(coords, columns=['lat', 'long']).to_csv('burglary-2015-lat-longs.csv')

In [19]:
m = folium.Map(location=[41.87871, -87.6298])

for coord in coords:
    folium.Marker(coord).add_to(m)

# HeatMap(np.unique(coords, axis=0).tolist()).add_to(m)

m